In [1]:
import tensorflow

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import pandas as pd

testdata = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/projects/test.csv")
traindata = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/projects/train.csv")
#submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/projects/gender_submission.csv")

testdata.head(3)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [4]:
traindata['Sex'].replace({'male':0,'female':1},inplace=True)
testdata['Sex'].replace({'male':0,'female':1},inplace=True)
traindata['Embarked'].replace({'S':0,'C':1,'Q':2},inplace=True)
testdata['Embarked'].replace({'S':0,'C':1,'Q':2},inplace=True)
traindata.head(3)

traindata['Parch'].value_counts()

testdata.isnull().sum()

#nan_rows = traindata[traindata['Embarked'].isnull()]

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
titles = set()
TitleD= {
    "Capt": 0,
    "Col": 0,
    "Major": 0,
    "Jonkheer": 1,
    "Don": 1,
    "Sir" : 1,
    "Dr": 0,
    "Rev": 1,
    "the Countess":1,
    "Mme": 2,
    "Mlle": 3,
    "Ms": 2,
    "Mr" : 4,
    "Mrs" : 2,
    "Miss" : 3,
    "Master" : 5,
    "Lady" : 1
}
for name in traindata['Name']:

    title=name.split(',')[1].split('.')[0].strip()
    for key in TitleD.keys():
      if title==key:
        title=TitleD[key]
    titles.add(name.split(',')[1].split('.')[0].strip())
    traindata['Name'].replace({name:title},inplace=True)
print(titles)

for name in testdata['Name']:

    title=name.split(',')[1].split('.')[0].strip()
    for key in TitleD.keys():
      if title==key:
        title=TitleD[key]
    titles.add(name.split(',')[1].split('.')[0].strip())
    testdata['Name'].replace({name:title},inplace=True)
testdata
testdata.loc[414,'Name']=2

{'Mlle', 'Col', 'Master', 'Mme', 'Lady', 'Don', 'Dr', 'Sir', 'Miss', 'Mrs', 'Mr', 'Ms', 'Capt', 'the Countess', 'Major', 'Rev', 'Jonkheer'}


In [6]:
import numpy as np
combined=pd.concat([traindata,testdata])
grouped_train = combined.groupby(['Sex','Pclass','Name'])
grouped_median_train = grouped_train.median()
print(grouped_train)
grouped_median_train = grouped_median_train.reset_index()[['Sex', 'Pclass', 'Name', 'Age']]
'''
mask= combined['Age'].isnull().any()
for i in combined[].iterrows():
  print(i)

'''
def fill_age(row):
    condition = (
        (grouped_median_train['Sex'] == row['Sex']) &
        (grouped_median_train['Name'] == row['Name']) &
        (grouped_median_train['Pclass'] == row['Pclass'])
    )
    return grouped_median_train[condition]['Age'].values[0]


def process_age():
    global combined
    # a function that fills the missing values of the Age variable
    combined['Age'] = combined.apply(lambda row: fill_age(row) if np.isnan(row['Age']) else row['Age'], axis=1)

    return combined


combined = process_age()
combined.head(30)
print(combined.loc[[28]])

TypeError: could not convert string to float: 'PC 17611'

In [ ]:
'''
import numpy as np

# Assuming 'combined' is your DataFrame and 'grouped_median_train' is another DataFrame containing median values

# Initialize lists
l = grouped_median_train['Sex']
m = grouped_median_train['Pclass']
n = grouped_median_train['Name']
k = grouped_median_train['Age']

# Loop through each row of grouped_median_train
for i in range(len(grouped_median_train)):
    # Get the current conditions
    sex_condition = combined['Sex'] == l[i]
    pclass_condition = combined['Pclass'] == m[i]
    name_condition = combined['Name'] == n[i]
    age_condition = np.isnan(combined['Age'])  # Check for NaN values in Age column

    # Combine conditions
    combined_condition = sex_condition & pclass_condition & name_condition & age_condition

    # Filter indices where all conditions are met
    indices = np.where(combined_condition)[0]
    print(indices)
    # Print the indices
print(combined.head(3))
print(combined.loc[[28]])
for i in indices:
  p=0
  #combined.loc[i]=k[p]
  p=p+1
  #print(combined.iloc[[i]])

    #combined[mask].apply(lambda x: )
     # fill=combined.Series([i,j,k],index=['Sex','Name','Pclass'])
'''

In [ ]:
new_trd=traindata.filter(['Sex','SibSp','Embarked','Pclass','Parch','Fare','Name'],axis=1)
new_ted=testdata.filter(['Sex','SibSp','Embarked','Pclass','Parch','Fare','Name'],axis=1)
new_trd

In [ ]:
print(new_ted.isnull().sum())
nan_rows = new_ted[new_ted['Fare'].isnull()]

new_trd.loc[61,'Embarked']=0
new_trd.loc[829,'Embarked']=0
new_ted.loc[152,'Fare']=1
print(nan_rows)
filt=(testdata['Name']=='Oliva y Ocana, Dona. Fermina')
testdata.loc[filt]


In [ ]:
import torch
from torch import nn
xtrain =torch.tensor(new_trd.values,dtype=torch.float32)
ytrain =torch.tensor(traindata['Survived'].values,dtype=torch.float32)
new_ted=new_ted.apply(pd.to_numeric)
xte =torch.tensor(new_ted.values,dtype=torch.float32)
print(new_ted.info())
print(new_trd.info())

In [ ]:
class titanic(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers=nn.Sequential(
        nn.Linear(in_features=7,out_features=64),
        nn.ReLU(),

        nn.Linear(in_features=64,out_features=16),
        nn.ReLU(),
        nn.Linear(16,1)
    )
  def forward(self,x:torch.tensor)->torch.tensor:
    return self.layers(x)



In [ ]:
m=titanic()
p=torch.optim.Adam(m.parameters(),lr=0.001)
l=nn.CrossEntropyLoss()
def accuracy(y_true,y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    print(acc)
    return acc

In [ ]:
trainx=torch.utils.data.TensorDataset(xtrain,ytrain)
dtr =torch.utils.data.DataLoader(trainx,batch_size=4, shuffle=False)
trainy=torch.utils.data.TensorDataset(xte)
dte =torch.utils.data.DataLoader(trainy,batch_size=4, shuffle=False)
train_features_batch, train_labels_batch = next(iter(dtr))
print(train_features_batch.shape, train_labels_batch.shape)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
epochs =100
trainloss=[]
epochcount=[]
b=[]
n=0
ctr=[]
cte=[]
correct=0
for epoch in range(epochs):
  trloss=0
  for batch , (xtr,ytr) in enumerate(dtr):

    m.train()
    a=m(xtr).squeeze()
    y=torch.round(torch.sigmoid(a))
    loss=l(a,ytr)
    p.zero_grad()
    loss.backward()
    p.step()

    trloss += loss
    correct += torch.eq(ytr, a).sum().item()
    n=n+1
    if batch%10==0:

     b.append(batch+(epoch+1)*230)
     #print(f"epoch={batch+(epoch+1)*230} , train loss={trloss} ")
     #accuracy(ytr,a)
      # torch.eq() calculates where two tensors are equal
     #print((n-correct)/n)
     epochcount.append(epoch)
     trainloss.append(trloss.detach().numpy())
  trloss /= len(dtr)



m.eval()

print(trloss)
print(len(xtr))
plt.plot(b, trainloss, label="Train loss")


In [ ]:

with torch.inference_mode():
    yp=m(xte).squeeze()
    ypr=torch.round(torch.sigmoid(yp))

    #submission['Survived'] = ypr
    #submission.to_csv('data/gender_submission.csv', index=False)
#sub=pd.DataFrame({'survived':[]})
number=[(i+1) for i in range(418)]

sub=pd.DataFrame(testdata['PassengerId'])


sub['Survived']=ypr
print(sub)
sub=sub.astype({'Survived':int})
sub.to_csv("/content/drive/MyDrive/Colab Notebooks/projects/gender_submission.csv", index=False)

print(sub)
sub['Survived'].value_counts()